In [1]:
import tensorflow as tf
# import open3d as o3d
import numpy as np
from models.pcn import *
from models.pccn import *

print(tf.__version__)

2.9.2


## Utils Function

In [2]:
def pprint(s, shape):
    print(s.center(35), ":",shape)

## PCN Development

In [14]:
# ids, inputs, npts, gt
npts = [20, 30, 55]
partial = tf.random.normal((1,105,3))
complete = tf.random.normal((3,16384,3))

model = PCN()
model.compile(optimizer="adam")
model.fit((1,partial,npts,complete), epochs=3)

# coarse, fine = PCN()()
# pprint("Output Tensor (aft decoder): coarse-->", coarse.shape)
# pprint("Output Tensor (aft decoder): fine-->", fine.shape)
# print()


ValueError: Failed to find data adapter that can handle input: (<class 'tuple'> containing values of types {"<class 'tensorflow.python.framework.ops.EagerTensor'>", '(<class \'list\'> containing values of types {"<class \'int\'>"})'}), <class 'NoneType'>

In [ ]:
def mlp_conv(inputs, channels):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv1D(channels[0], 1, activation='relu'))
    model.add(tf.keras.layers.Conv1D(channels[1], 1, activation='relu'))
    outputs = model(inputs)
    return outputs

def mlp(inputs, channels):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(channels[0], activation='relu'))
    model.add(tf.keras.layers.Dense(channels[1], activation='relu'))
    model.add(tf.keras.layers.Dense(channels[2], activation='relu'))
    outputs = model(inputs)
    return outputs

def point_maxpool(inputs, npts, keepdims=False):
    outputs = [tf.reduce_max(f, axis=1, keepdims=keepdims)
        for f in tf.split(inputs, npts, axis=1)]
    return tf.concat(outputs, axis=0)


def point_unpool(inputs, npts):
    inputs = tf.split(inputs, inputs.shape[0], axis=0)
    outputs = [tf.tile(f, [1, npts[i], 1]) for i,f in enumerate(inputs)]
    return tf.concat(outputs, axis=1)

# npts = [1024,1024,1024]
# x = tf.random.normal((3,1024,3))
# pprint("Input Tensor", x.shape)
# x = tf.reshape(x, [1, x.shape[1]*x.shape[0], 3])
npts = [20, 30, 55]
x = tf.random.normal((1,105,3))
pprint("Input Tensor (aft reshaping)", x.shape)
features = mlp_conv(x,[64,128])
pprint("Output Tensor (aft Conv1D)", features.shape)

features_1 = point_maxpool(features, npts, keepdims=1)
pprint("Output Tensor (aft maxpool)", features_1.shape)
features_global = point_unpool(features_1, npts)
pprint("Output Tensor (aft unpool)", features_global.shape)

features = tf.concat([features, features_global], axis=2)
pprint("Output Tensor (aft concat)", features.shape)

features = mlp_conv(features,[512,1024])
pprint("Output Tensor (aft 2nd conv)", features.shape)
features = point_maxpool(features,npts)
pprint("Output Tensor (aft 2nd maxpool)", features.shape)


coarse = mlp(features,[1024,1024, 1024*3])
pprint("Output Tensor (aft MLP)", coarse.shape)

coarse = tf.reshape(coarse, [-1, 1024, 3])
pprint("Output Tensor (aft reshaping)", coarse.shape)

"""
    Input Tensor (aft reshaping)    : (1, 105, 3)
     Output Tensor (aft Conv1D)     : (1, 105, 128)
    Output Tensor (aft maxpool)     : (3, 1, 128)
     Output Tensor (aft unpool)     : (1, 105, 128)
     Output Tensor (aft concat)     : (1, 105, 256)
    Output Tensor (aft 2nd conv)    : (1, 105, 1024)
  Output Tensor (aft 2nd maxpool)   : (3, 1024)
      Output Tensor (aft MLP)       : (3, 3072)
   Output Tensor (aft reshaping)    : (3, 1024, 3)
"""


## PCCN Development

In [ ]:
input_shape = (8, np.random.randint(100), 3, 1)
num_output = 8192
bs = input_shape[0]
num_pts = input_shape[1]

x = tf.random.normal(input_shape)
print("Input Tensor (raw):",x.shape)
x = PointNet(x.shape, 128)(x)
print("Output Tensor (Encoder):",x.shape)
x = MLP()(x)
print("Output Tensor (Decoder):",x.shape)

x = tf.reshape(x, [bs, 8192, 3])
print("Final Output:", x.shape)

In [ ]:
input_shape = (12, 189, 3,1)
x = tf.random.normal(input_shape)
print(x)

In [ ]:
input_shape = (12, 189, 3, 1)
bs = input_shape[0]
num_pts = input_shape[1]

# Point functions (MLP implemented as conv2d)
# By default, Keras version of Conv2D has: strides=[1,1], padding="valid" , data_format="channels_last"
x = tf.random.normal(input_shape)
x = tf.keras.layers.Conv2D(64, [1,3], activation='relu')(x)
print(x.shape)
x = tf.keras.layers.Conv2D(64, [1,1],activation='relu')(x)
print(x.shape)
x = tf.keras.layers.Conv2D(64, [1,1], activation='relu')(x)
print(x.shape)
x = tf.keras.layers.Conv2D(128, [1,1], activation='relu')(x)
print(x.shape)
x = tf.keras.layers.Conv2D(1024, [1,1], activation='relu')(x)
print(x.shape)

# Symmetric function: max pooling
x = tf.keras.layers.MaxPool2D(pool_size=[num_pts, 1], strides=(2, 2), padding='valid')(x)
print(x.shape)

# MLP on global point cloud vector
x = tf.reshape(x, [bs, -1])
print(x.shape)
x = tf.keras.layers.Dense(1024,activation='relu')(x)
print(x.shape)
x = tf.keras.layers.Dense(512,activation='relu')(x)
print(x.shape)
x = tf.keras.layers.Dense(256,activation='relu')(x)
print(x.shape)

mean, logvar = tf.split(x, num_or_size_splits=2, axis=1)
print("Mean:",mean.shape, " Log(Var):",logvar.shape)